## The Battle of Neighborhoods (Week 2)

### By Ritu

I  will use the Notebook to build the code to scrape the following web page, https://www.mapsofindia.com/pincode/india/karnataka/bangalore/, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

### Step 1: Import Libraries

In [1]:
!pip install beautifulsoup4 -U
!pip install lxml -U
!pip install requests -U
!pip install html5lib -U
!pip install urllib3 -U
!pip install geocoder -U

Requirement already up-to-date: beautifulsoup4 in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.8.1)
Requirement already up-to-date: lxml in /opt/conda/envs/Python36/lib/python3.6/site-packages (4.4.2)
Requirement already up-to-date: requests in /opt/conda/envs/Python36/lib/python3.6/site-packages (2.22.0)
Requirement already up-to-date: html5lib in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.0.1)
Requirement already up-to-date: urllib3 in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.25.7)
Requirement already up-to-date: geocoder in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.38.1)


In [2]:
import html.parser
from bs4 import BeautifulSoup
import requests
import urllib3.request
import pandas as pd
import numpy as np

### Step 2: Scraping the table from the web

I will use the Notebook to build the code to scrape the following web page, https://www.mapsofindia.com/pincode/india/karnataka/bangalore/, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe.

In [3]:
from lxml import etree
url = 'https://www.mapsofindia.com/pincode/india/karnataka/bangalore/'
response = requests.get('https://www.mapsofindia.com/pincode/india/karnataka/bangalore/').content
df_list = pd.read_html(response)
df = df_list[-1]
df.to_csv('bangaloredata.csv')

df.shape

(295, 4)

In [4]:
df.head(10)

,Pincode Details,Pincode Details.1,Pincode Details.2,Pincode Details.3
0,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
6,Amruthahalli,560092,Karnataka,Bangalore
7,Anandnagar,560024,Karnataka,Bangalore
8,Anekal,562106,Karnataka,Bangalore
9,Anekalbazar,562106,Karnataka,Bangalore


### Step 3: Create DataFrame

The dataframe will consist of four columns: PostalCode, State, District,Location.

In [5]:
df.columns = df.iloc[0]
df.columns.name=None
df.drop(0, axis=0, inplace=True)
df.head(10)

,Location,Pincode,State,District
1,A F station yelahanka,560063,Karnataka,Bangalore
2,Adugodi,560030,Karnataka,Bangalore
3,Agara,560034,Karnataka,Bangalore
4,Agram,560007,Karnataka,Bangalore
5,Air Force hospital,560007,Karnataka,Bangalore
6,Amruthahalli,560092,Karnataka,Bangalore
7,Anandnagar,560024,Karnataka,Bangalore
8,Anekal,562106,Karnataka,Bangalore
9,Anekalbazar,562106,Karnataka,Bangalore
10,Arabic College,560045,Karnataka,Bangalore


In [6]:
df.rename(columns={"Pincode": "PostalCode"}, inplace=True)
df_grouped = df.groupby(["PostalCode", "State", "District"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped
print(df_grouped[['PostalCode','State','District','Location']])

   PostalCode      State   District  \
0      560001  Karnataka  Bangalore   
1      560002  Karnataka  Bangalore   
2      560003  Karnataka  Bangalore   
3      560004  Karnataka  Bangalore   
4      560005  Karnataka  Bangalore   
5      560006  Karnataka  Bangalore   
6      560007  Karnataka  Bangalore   
7      560008  Karnataka  Bangalore   
8      560009  Karnataka  Bangalore   
9      560010  Karnataka  Bangalore   
10     560011  Karnataka  Bangalore   
11     560012  Karnataka  Bangalore   
12     560013  Karnataka  Bangalore   
13     560014  Karnataka  Bangalore   
14     560015  Karnataka  Bangalore   
15     560016  Karnataka  Bangalore   
16     560017  Karnataka  Bangalore   
17     560018  Karnataka  Bangalore   
18     560019  Karnataka  Bangalore   
19     560020  Karnataka  Bangalore   
20     560021  Karnataka  Bangalore   
21     560022  Karnataka  Bangalore   
22     560023  Karnataka  Bangalore   
23     560024  Karnataka  Bangalore   
24     560025  Karnataka 

In [7]:
df_grouped.head(10)

,PostalCode,State,District,Location
0,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu..."
1,560002,Karnataka,Bangalore,"Avenue Road, Bangalore City, Bangalore Corpora..."
2,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,..."
3,560004,Karnataka,Bangalore,"Basavanagudi, Lalbagh West, Mavalli, Pasmpamah..."
4,560005,Karnataka,Bangalore,"Fraser Town, Jeevanahalli"
5,560006,Karnataka,Bangalore,"J.C.nagar, Training Command iaf"
6,560007,Karnataka,Bangalore,"Agram, Air Force hospital"
7,560008,Karnataka,Bangalore,"H.A.l ii stage, Hulsur Bazaar, Someswarapura"
8,560009,Karnataka,Bangalore,"Bangalore Dist offices bldg, K. g. road, Subha..."
9,560010,Karnataka,Bangalore,"Bhashyam Circle, Industrial Estate, Rajajinaga..."


In [7]:
df_grouped.shape

(99, 4)

In [8]:
df_grouped['PostalCode']=df_grouped['PostalCode'].astype(int)

### Step 4: Use the csv file to create DataFrame

This dataframe will contain the geospatial data of India obtained from the website https://data.gov.in/sites/default/files/all_india_PO_list_without_APS_offices_ver2_lat_long.csv

In [9]:
import ibm_db
dsn_driver = "IBM DB2 ODBC DRIVER"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net" # e.g.: "awh-yp-small03.services.dal.bluemix.net"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"
dsn_uid = "rzj00913"        # e.g. "dash104434"
dsn_pwd = "jdsnvp9pg0t3hr+q"

dsn = (
    "DRIVER={{IBM DB2 ODBC DRIVER}};"
    "DATABASE={0};"
    "HOSTNAME={1};"
    "PORT={2};"
    "PROTOCOL=TCPIP;"
    "UID={3};"
    "PWD={4};").format(dsn_database, dsn_hostname, dsn_port, dsn_uid, dsn_pwd)

conn = ibm_db.connect(dsn, "", "")

df1 = "SELECT * FROM LOCATION_DATA_INDIA;"

stmt = ibm_db.exec_immediate(conn, df1)
ibm_db.fetch_both(stmt)

{'officename': 'Achalapur B.O',
 0: 'Achalapur B.O',
 'pincode': 504273,
 1: 504273,
 'officeType': 'B.O',
 2: 'B.O',
 'Deliverystatus': 'Delivery',
 3: 'Delivery',
 'divisionname': 'Adilabad',
 4: 'Adilabad',
 'regionname': 'Hyderabad',
 5: 'Hyderabad',
 'circlename': 'Andhra Pradesh',
 6: 'Andhra Pradesh',
 'Taluk': 'Asifabad',
 7: 'Asifabad',
 'Districtname': 'Adilabad',
 8: 'Adilabad',
 'statename': 'TELANGANA',
 9: 'TELANGANA',
 'Telephone': 'NA',
 10: 'NA',
 'Related_Suboffice': 'Rechini S.O',
 11: 'Rechini S.O',
 'Related_Headoffice': 'Mancherial H.O',
 12: 'Mancherial H.O',
 'longitude': 'NA',
 13: 'NA',
 'latitude': 'NA',
 14: 'NA'}

In [10]:
import pandas
import ibm_db_dbi
pconn = ibm_db_dbi.Connection(conn)
bangalore_coord = pandas.read_sql('SELECT * FROM LOCATION_DATA_INDIA', pconn)

In [11]:
bangalore_coord.drop(["officename", "officeType","Deliverystatus","divisionname","regionname","circlename", "Taluk","Districtname", "statename", "Telephone","Related_Suboffice","Related_Headoffice"], axis=1, inplace=True)
bangalore_coord.rename(columns={"pincode": "PostalCode"}, inplace=True)

bangalore_coord.head(10)

,PostalCode,longitude,latitude
0,504273,NA,NA
1,504293,NA,NA
2,504307,NA,NA
3,504001,NA,NA
4,504001,NA,NA
5,504251,NA,NA
6,504273,NA,NA
7,504251,NA,NA
8,504309,NA,NA
9,504201,NA,NA


In [12]:
bangalore_coord["longitude"] = bangalore_coord["longitude"].replace('NA', np.nan)
bangalore_coord = bangalore_coord.dropna(axis=0, subset=["longitude"])
bangalore_coord.head(10)

,PostalCode,longitude,latitude
5154,500016,78.4468,17.4672
52180,560045,77.6206,13.0291
52187,560103,77.676,12.9298
52201,560071,77.6359,12.9611
52204,560077,77.6293,13.0681
52206,560005,77.6164,13.0005
52207,560065,77.5784,13.0781
52210,560008,77.6427,12.9686
52213,560001,77.5915,12.9773
52223,560075,77.6535,12.9695


In [13]:
bangalore_coord['PostalCode']=bangalore_coord['PostalCode'].astype(int)

### Step 5: Merge both the DataFrames to create one Final DataFrame

Here I will merge the Bangalore Postalcode data and coordinates data to obtain final location data for Bangalore, Karnataka.

In [14]:
bangalore_tab = df_grouped.merge(bangalore_coord, on="PostalCode", how="left")
bangalore_tab.head()

,PostalCode,State,District,Location,longitude,latitude
0,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5915,12.9773
1,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.61,12.9739
2,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5955,12.9814
3,560002,Karnataka,Bangalore,"Avenue Road, Bangalore City, Bangalore Corpora...",NaN,NaN
4,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5709,13.0002


In [15]:
bangalore_tab.shape

(114, 6)

In [16]:
bangalore_tab = bangalore_tab.dropna(axis=0, subset=["longitude"])
bangalore_tab

,PostalCode,State,District,Location,longitude,latitude
0,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5915,12.9773
1,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.61,12.9739
2,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5955,12.9814
4,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5709,13.0002
5,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5819,12.9993
6,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.574,12.9968
7,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5769,13.004
8,560004,Karnataka,Bangalore,"Basavanagudi, Lalbagh West, Mavalli, Pasmpamah...",77.5741,12.9446
9,560005,Karnataka,Bangalore,"Fraser Town, Jeevanahalli",77.6164,13.0005
12,560008,Karnataka,Bangalore,"H.A.l ii stage, Hulsur Bazaar, Someswarapura",77.6427,12.9686


In [17]:
bangalore_tab.shape

(54, 6)

### Step 5: Clustering the neighborhoods in Bangalore

I will be using the Foursquare API to explore neighborhoods in Bangalore. The Foursquare explore function will be used to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. Here I will be using k-means clustering for analysising the data of neighborhoods in Bangalore. Finally, use the Folium library to visualize the neighborhoods in Bangalore and their emerging clusters. For that I will download the required libraries and dependencies.

In [18]:
!conda install -c conda-forge folium=0.5.0 --yes
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim 
GeoLocator = Nominatim(user_agent='My-IBMNotebook')

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [19]:
bangalore_tab.to_csv('bangalore_tab.csv')
bangalore_tab_csv = "bangalore_tab.csv"

In [20]:
bangalore_neighborhoods = pd.read_csv(bangalore_tab_csv)
bangalore_neighborhoods.head()

,Unnamed: 0,PostalCode,State,District,Location,longitude,latitude
0,0,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5915,12.9773
1,1,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.6100,12.9739
2,2,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5955,12.9814
3,4,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5709,13.0002
4,5,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5819,12.9993


In [21]:
bangalore_latitude = 12.9716; bangalore_longitude = 77.5946 #Bangalore's latitude and longtitude are manually extracted via google search
map_bangalore = folium.Map(location = [bangalore_latitude, bangalore_longitude], zoom_start = 15)
for lat, lng, Location in zip(bangalore_neighborhoods['longitude'], bangalore_neighborhoods['latitude'], bangalore_neighborhoods['Location']):
    label = format(Location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=7,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_bangalore) 
map_bangalore

In [22]:
CLIENT_ID = '5CGGZXVKML4QGFGWGITB234HSQQCLIBJHP4QHDNBCKJCYLQX' 
CLIENT_SECRET = 'BWBCFYUBB4RFSHXM2W0QCMKPA3UWFGBLAGQSI30EMMDSRMZF'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5CGGZXVKML4QGFGWGITB234HSQQCLIBJHP4QHDNBCKJCYLQX
CLIENT_SECRET:BWBCFYUBB4RFSHXM2W0QCMKPA3UWFGBLAGQSI30EMMDSRMZF


In [23]:
bangalore_data = bangalore_neighborhoods[bangalore_neighborhoods['District'] == 'Bangalore'].reset_index(drop=True)
bangalore_data.head()

,Unnamed: 0,PostalCode,State,District,Location,longitude,latitude
0,0,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5915,12.9773
1,1,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.6100,12.9739
2,2,560001,Karnataka,Bangalore,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",77.5955,12.9814
3,4,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5709,13.0002
4,5,560003,Karnataka,Bangalore,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",77.5819,12.9993


#### Let's get the top 100 venues in the neighborhood 'Mahatma Gandhi road', from Scarborough within 500 meters radius

In [24]:
neighborhood_latitude = bangalore_data.loc[0, 'latitude']
neighborhood_longitude = bangalore_data.loc[0, 'longitude']

neighborhood_name = bangalore_data.loc[0, 'Location']

print('Latitude and longitude values of "{}" are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of "Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha" are 12.9773, 77.5915.


In [25]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [26]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dec7d9578a484001bf4d82e'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bangalore',
  'headerFullLocation': 'Bangalore',
  'headerLocationGranularity': 'city',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 12.981800004500004,
    'lng': 77.59610932882987},
   'sw': {'lat': 12.972799995499996, 'lng': 77.58689067117012}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bbc4513e45295217db855a4',
       'name': 'Cubbon Park',
       'location': {'address': 'M G Road',
        'lat': 12.977042175203783,
        'lng': 77.59527683258057,
        'labeledLatLngs': [{'label': 'display',
          'lat': 12.977042175203783,
          'lng

In [27]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [28]:
import json 
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']  
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

,name,categories,lat,lng
0,Cubbon Park,Park,12.977042,77.595277
1,Vidhana Soudha,Capitol Building,12.979027,77.591881
2,KSLTA Tennis stadium,Tennis Stadium,12.975094,77.594439
3,Cubbon Park Library Play Ground,Dog Run,12.974522,77.590236


In [29]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [30]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [31]:
bangalore_venues = getNearbyVenues(names=bangalore_data['Location'],
                                   latitudes=bangalore_data['latitude'],
                                   longitudes=bangalore_data['longitude']
                                  )

Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha
Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha
Bangalore Bazaar, Bangalore., Brigade Road, Cubban Road, Dr. ambedkar veedhi, Highcourt, Legislators Home, Mahatma Gandhi road, Narayan Pillai street, Rajbhavan, Vidhana Soudha
Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn
Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn
Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn
Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn
Basavanagudi, Lalbagh West, Mavalli, Pasmpamahakavi Road, Shankar

In [32]:
bangalore_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Cubbon Park,12.977042,77.595277,Park
1,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Vidhana Soudha,12.979027,77.591881,Capitol Building
2,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,KSLTA Tennis stadium,12.975094,77.594439,Tennis Stadium
3,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Cubbon Park Library Play Ground,12.974522,77.590236,Dog Run
4,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Nagarjuna,12.973145,77.609263,Andhra Restaurant
5,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Starbucks,12.974436,77.607308,Coffee Shop
6,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Chung Wah,12.974099,77.609386,Chinese Restaurant
7,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Corner House,12.973650,77.609362,Dessert Shop
8,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Matteo,12.974496,77.607115,Café
9,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Art of Delight,12.973198,77.609254,Ice Cream Shop


In [33]:
bangalore_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Agara, Kendriya Sadan, Koramangala, Koramangala I block, St. john's medical college",6,6,6,6,6,6
"Amruthahalli, Kodigehalli, Sahakaranagar P.o",17,17,17,17,17,17
"Arabic College, Devarjeevanahalli, Nagavara, Ramakrishna Hegde nagar, Venkateshapura",2,2,2,2,2,2
"Aranya Bhavan, Malleswaram, Palace Guttahalli, Swimming Pool extn, Venkatarangapura, Vyalikaval Extn",70,70,70,70,70,70
"Ashoknagar, Banashankari, Dasarahalli, State Bank of mysore colony",10,10,10,10,10,10
"Attur, Bsf Campus yelahanka, Crpf Campus yelahanka, Jakkur, Kendriya Vihar, Rajanakunte, Singanayakanahalli, Wheel And axle plant, Yelahanka, Yelahanka Satellite town",8,8,8,8,8,8
"Avani Sringeri mutt, Basaveswaranagar Ii stage, Mahalakshmipuram Layout",12,12,12,12,12,12
"B Sk ii stage, Jayanagar West, Padmanabhnagar, Yediyur",11,11,11,11,11,11
"Bangalore Air port, Nal, Vimapura",33,33,33,33,33,33


In [34]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

There are 143 uniques categories.


#### Analyze Each Neighborhood

In [35]:
ban_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

ban_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

fixed_columns = [ban_onehot.columns[-1]] + list(ban_onehot.columns[:-1])
ban_onehot = ban_onehot[fixed_columns]

ban_onehot.head()

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Paella Restaurant,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pub,Punjabi Restaurant,Resort,Restaurant,Road,Roof Deck,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Women's Store
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
ban_onehot.shape

(839, 144)

#### Let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [37]:
ban_grouped = ban_onehot.groupby('Neighborhood').mean().reset_index()
ban_grouped.head()

,Neighborhood,Accessories Store,Airport Terminal,American Restaurant,Andhra Restaurant,Antique Shop,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Beer Bar,Bengali Restaurant,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Candy Store,Capitol Building,Chaat Place,Chettinad Restaurant,Chinese Restaurant,Chocolate Shop,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish Market,Flea Market,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Gastropub,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Hookah Bar,Hospital,Hot Dog Joint,Hotel,Hotel Bar,Hyderabadi Restaurant,Ice Cream Shop,Indian Restaurant,Indian Sweet Shop,Indie Movie Theater,Intersection,Italian Restaurant,Jewelry Store,Juice Bar,Karnataka Restaurant,Kerala Restaurant,Lake,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Multicuisine Indian Restaurant,Multiplex,Music Venue,Nightclub,North Indian Restaurant,Office,Optical Shop,Paella Restaurant,Pakistani Restaurant,Park,Performing Arts Venue,Persian Restaurant,Pharmacy,Pizza Place,Playground,Plaza,Pub,Punjabi Restaurant,Resort,Restaurant,Road,Roof Deck,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soccer Field,South Indian Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Szechuan Restaurant,Tea Room,Tennis Stadium,Tex-Mex Restaurant,Thai Restaurant,Tibetan Restaurant,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Watch Shop,Women's Store
0,"Agara, Kendriya Sadan, Koramangala, Koramangal...",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.166667,0.166667,0.0,0.000000,0.166667,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.166667,0.0,0.000000,0.0,0.0,0.000000,0.166667,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
1,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.058824,0.058824,0.0,0.0,0.0,0.0,0.058824,0.0,0.0,0.0,0.0,0.058824,0.0,0.000000,0.0,0.058824,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.058824,0.0,0.117647,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.117647,0.176471,0.0,0.0,0.0,0.058824,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.058824,0.0,0.000000,0.0,0.0,0.058824,0.000000,0.0,0.0,0.058824,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000
2,"Arabic College, Devarjeevanahalli, Nagavara, R...",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0

#### Get top 10 venues per neighborhood

In [38]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [39]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = ban_grouped['Neighborhood']

for ind in np.arange(ban_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(ban_grouped.iloc[ind, :], num_top_venues)

In [40]:
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Agara, Kendriya Sadan, Koramangala, Koramangal...",Hotel Bar,Breakfast Spot,Indian Restaurant,Pizza Place,Restaurant,Hotel,Arts & Crafts Store,Fish Market,Dance Studio,Deli / Bodega
1,"Amruthahalli, Kodigehalli, Sahakaranagar P.o",Indian Restaurant,Ice Cream Shop,Fast Food Restaurant,Electronics Store,Café,Coffee Shop,Bubble Tea Shop,Brewery,Pizza Place,Chinese Restaurant
2,"Arabic College, Devarjeevanahalli, Nagavara, R...",Indian Restaurant,Gym / Fitness Center,Women's Store,Diner,Farmers Market,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
3,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",Indian Restaurant,Department Store,Vegetarian / Vegan Restaurant,Café,Pizza Place,Ice Cream Shop,Clothing Store,Coffee Shop,Jewelry Store,Snack Place
4,"Ashoknagar, Banashankari, Dasarahalli, State B...",Indian Restaurant,Juice Bar,Breakfast Spot,Park,Plaza,Athletics & Sports,Indie Movie Theater,Szechuan Restaurant,Women's Store,Eastern European Restaurant
5,"Attur, Bsf Campus yelahanka, Crpf Campus yelah...",Clothing Store,Shopping Mall,Dessert Shop,Multiplex,Café,Bus Station,Seafood Restaurant,Farmers Market,Gastropub,Garden
6,"Avani Sringeri mutt, Basaveswaranagar Ii stage...",Indian Restaurant,Snack Place,Asian Restaurant,Bus Line,Light Rail Station,Food Truck,Beer Bar,Bakery,Park,Tram Station
7,"B Sk ii stage, Jayanagar West, Padmanabhnagar,...",Ice Cream Shop,Coffee Shop,Café,Snack Place,Park,Fast Food Restaurant,Mediterranean Restaurant,Indian Restaurant,Pizza Place,Convenience Store
8,"Bangalore Air port, Nal, Vimapura",Café,Indian Restaurant,Coffee Shop,Middle Eastern Restaurant,Hotel,Bar,Shopping Mall,French Restaurant,Chinese Restaurant,Pizza Place
9,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",Café,Indian Restaurant,Pub,Hotel,Chinese Restaurant,Donut Shop,Clothing Store,Pizza Place,Brewery,Fast Food Restaurant


#### Cluster Neighborhoods:

##### Assigning weights to some of the categories:

In [41]:
k = bangalore_venues.copy(deep = True)
weights_dict={'Gym / Fitness Center':2,'Gym':2,'Fitness Center':2,'Playground':3,'Park':3,'Sports Bar':4}
data = bangalore_venues['Venue Category']
allVenues = data
weights = []
for i in allVenues:
    if i in weights_dict.keys():
        weights.append(weights_dict[i])
    else :
        weights.append(0)
bangalore_venues['weights'] = weights;
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Cubbon Park,12.977042,77.595277,Park,3
1,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Vidhana Soudha,12.979027,77.591881,Capitol Building,0
2,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,KSLTA Tennis stadium,12.975094,77.594439,Tennis Stadium,0
3,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Cubbon Park Library Play Ground,12.974522,77.590236,Dog Run,0
4,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Nagarjuna,12.973145,77.609263,Andhra Restaurant,0


In [42]:
bangalore_venues.drop(bangalore_venues[bangalore_venues.weights < 1.0].index, inplace=True)
bangalore_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,weights
0,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9773,77.5915,Cubbon Park,12.977042,77.595277,Park,3
24,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9739,77.6100,Fitness First,12.971153,77.607010,Gym,2
74,"Bangalore Bazaar, Bangalore., Brigade Road, Cu...",12.9814,77.5955,Cubbon Park,12.977042,77.595277,Park,3
145,"Aranya Bhavan, Malleswaram, Palace Guttahalli,...",13.0040,77.5769,Fitness One,13.001221,77.577104,Gym,2
186,"Fraser Town, Jeevanahalli",13.0005,77.6164,Richard's Park,13.004262,77.617396,Park,3


#### Copying only the relevants columns like city and weights to group all the venues by Neighborhood and calculating the means for each Neighborhood

In [44]:
venues_weights = bangalore_venues[['Neighborhood','weights']].copy()
venues_weights_means = venues_weights.groupby(['Neighborhood']).mean()
venues_weights_means = venues_weights_means.reset_index(drop=False)
venues_weights_means.head

<bound method NDFrame.head of                                          Neighborhood   weights
0   Arabic College, Devarjeevanahalli, Nagavara, R...  2.000000
1   Aranya Bhavan, Malleswaram, Palace Guttahalli,...  2.000000
2   Ashoknagar, Banashankari, Dasarahalli, State B...  3.000000
3   Avani Sringeri mutt, Basaveswaranagar Ii stage...  3.000000
4   B Sk ii stage, Jayanagar West, Padmanabhnagar,...  3.000000
5                   Bangalore Air port, Nal, Vimapura  4.000000
6   Bangalore Bazaar, Bangalore., Brigade Road, Cu...  2.666667
7   Bannerghatta Road, Hulimavu, Mico Layout, Moun...  2.500000
8                                           Bellandur  2.000000
9                 Bommanahalli, Hongasandra, Madivala  3.000000
10                      Dharmaram College, Taverekere  3.000000
11                          Fraser Town, Jeevanahalli  2.500000
12  Govindapalya, H M t, Jalahalli, Jalahalli Village  3.000000
13       H.A.l ii stage, Hulsur Bazaar, Someswarapura  3.000000
14        

#### Best scored Neighborhood

In [50]:
from scipy import stats
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
#Standardize
clmns = ['weights','Venue Latitude', 'Venue Longitude']
df_tr_std = stats.zscore(bangalore_venues[clmns])
#Cluster the data
kmeans = KMeans(n_clusters=3, random_state=0).fit(df_tr_std)
labels = kmeans.labels_
bangalore_venues['clusters'] = labels
#Add the column into our list
clmns.extend(['clusters'])

In [48]:
kframe = bangalore_venues[clmns].groupby(['weights']).mean()
kframe = kframe.reset_index(drop = False)
kframe.head()

,weights,Venue Latitude,Venue Longitude,clusters
0,2,12.959639,77.610145,0.250000
1,3,12.966318,77.588792,1.733333
2,4,12.965177,77.643796,2.000000


In [51]:
bangalore_latitude = 12.965177; bangalore_longitude = 77.643796
final_map = folium.Map(location=[bangalore_latitude, bangalore_longitude], zoom_start=15) # generate map centred around Jersey city


# add prefered location in the City as a green circle mark
folium.features.CircleMarker(
    [bangalore_latitude, bangalore_longitude],
    radius=50,
    popup='Gym can be opened within this circle',
    fill=True,
    color='green',
    fill_color='green',
    fill_opacity=0.6
    ).add_to(final_map)
final_map